# pyPCGA stwave inversion example

In [ ]:
%matplotlib inline

- import relevant python packages after installing pyPCGA
- stwave.py includes python wrapper to stwave model

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib
from scipy.io import savemat, loadmat
import numpy as np
import stwave as st
from pyPCGA import PCGA
import math
import datetime as dt
import os

In [ ]:
matplotlib.rcParams['font.size'] = 16

In [ ]:
basedir = os.getcwd()
workdir=os.path.join(basedir,'results','duck','run01')

In [ ]:
os.chdir(workdir)

- model domain and discretization

In [ ]:
N = np.array([110,83])
m = np.prod(N) 
dx = np.array([5.,5.])
xmin = np.array([0. + dx[0]/2., 0. + dx[1]/2.])
xmax = np.array([110.*5. - dx[0]/2., 83.*5. - dx[1]/2.])

- covariance kernel and scale parameters following Hojat's CSKF paper

In [ ]:
prior_std = 1.5
prior_cov_scale = np.array([18.*5., 18.*5.])
def kernel(r): return (prior_std**2)*np.exp(-r**2)

- grid coordinates for plotting purposes

In [ ]:
x = np.linspace(0. + dx[0]/2., 110*5 - dx[0]/2., N[0])
y = np.linspace(0. + dx[1]/2., 83*5 - dx[0]/2., N[1])
XX, YY = np.meshgrid(x, y)
pts = np.hstack((XX.ravel()[:,np.newaxis], YY.ravel()[:,np.newaxis]))

- load data, true field is optional

In [ ]:
obs = np.loadtxt('obs.txt')
s_true = np.loadtxt('true_depth.txt')

- define domain extent, discretization and measurement collection time

In [ ]:
nx = 110
ny = 83
Lx = 550
Ly = 415
x0, y0 = (62.0, 568.0)
t1 = dt.datetime(2015, 10, 07, 20, 00)
t2 = dt.datetime(2015, 10, 07, 21, 00)

stwave_params = {'nx': nx, 'ny': ny, 'Lx': Lx, 'Ly': Ly, 'x0': x0, 'y0': y0, 't1': t1, 't2': t2,
          'offline_dataloc': "./input_files/8m-array_2015100718_2015100722.nc"}

- parameters for the simulation

In [ ]:
params = {'R':(0.1)**2, 'n_pc':50,
          'maxiter':10, 'restol':0.01,
          'matvec':'FFT','xmin':xmin, 'xmax':xmax, 'N':N,
          'prior_std':prior_std,'prior_cov_scale':prior_cov_scale,
          'kernel':kernel, 'post_cov':"diag",
          'precond':True, 'LM': True,
          'parallel':True, 'linesearch' : True,
          'forward_model_verbose': False, 'verbose': False,
          'iter_save': True}

- initial guess

In [ ]:
s_init = np.mean(s_true)*np.ones((m,1))

- initialize PCGA object

- run get inversion results

In [ ]:
s_hat     = np.loadtxt('shat3.txt')
simul_obs = np.loadtxt('simulobs3.txt')
post_diagv= np.loadtxt('postv.txt')
error = s_true2d-s_hat2d

In [ ]:
# converting to 2d array for plotting
s_hat2d = s_hat.reshape(N[1],N[0])
s_true2d = s_true.reshape(N[1],N[0])
post_diagv[post_diagv <0.] = 0. # just in case
post_std = np.sqrt(post_diagv)
post_std2d = post_std.reshape(N[1],N[0])

- plot results

In [ ]:
minv = s_true.min()
maxv = s_true.max()

fig, axes = plt.subplots(1,2, figsize=(15,5))
plt.suptitle('prior var.: (%g)^2, n_pc : %d' % (prior_std,params['n_pc']))
im = axes[0].imshow(np.flipud(np.fliplr(-s_true2d)), extent=[0, 110, 0, 83], vmin=-7., vmax=0., cmap=plt.get_cmap('jet'))
axes[0].set_title('(a) True', loc='left')
axes[0].set_aspect('equal')
axes[0].set_xlabel('Offshore distance (px)')
axes[0].set_ylabel('Alongshore distance (px)')
axes[1].imshow(np.flipud(np.fliplr(-s_hat2d)), extent=[0, 110, 0, 83], vmin=-7., vmax=0., cmap=plt.get_cmap('jet'))
axes[1].set_title('(b) Estimate', loc='left')
axes[1].set_xlabel('Offshore distance (px)')
axes[1].set_aspect('equal')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)


- plot uncertainty

In [ ]:
fig = plt.figure()
im = plt.imshow(np.flipud(np.fliplr(post_std2d)), extent=[0, 110, 0, 83], cmap=plt.get_cmap('jet'))
plt.title('Uncertainty (std)', loc='left')
plt.xlabel('Offshore distance (px)')
plt.ylabel('Alongshore distance (px)')
plt.gca().set_aspect('equal', adjustable='box')
fig.colorbar(im)


- plot transect at y = 25 px and 45 px

In [ ]:
def plot_1x1(nx,ny,dx,dy,s0,fig=None,tit0='Estimate',
             vmin=[-7.],vmax=[0.],cmap_type='jet'):
    if fig is None:
        fig,axes = plt.subplots(1,1, figsize=(15,5))
    else:
        axes = fig.add_subplot(1,1,1,adjustable='box',aspect=1)


    linex = np.arange(1,nx+1)*dx
    liney = np.arange(1,ny+1)*dy
    im0 = axes.imshow(np.flipud(np.fliplr(-s0)), extent=[0, linex[-1], 0, liney[-1]], vmin=vmin[0], vmax=vmax[0], 
                        cmap=plt.get_cmap(cmap_type))
    axes.set_title('(a) {0}'.format(tit0), loc='left')
    axes.set_aspect('equal')
    axes.set_xlabel('Offshore distance [m]')
    axes.set_ylabel('Alongshore distance [m]')
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.65, 0.15, 0.05, 0.7])
    fig.colorbar(im0, cax=cbar_ax)


    return fig,axes



In [ ]:
def plot_2x1(nx,ny,dx,dy,s0,s1,fig=None,tit0='True',tit1='Estimate',
             vmin=[-7.,-7.],vmax=[0.,0.],cmap_type='jet',colorbar_flag=0):
    if fig is None:
        fig,axes = plt.subplots(1,2, figsize=(15,5))
    else:
        axes = []
        axes.append(fig.add_subplot(1,2,1,adjustable='box',aspect=1))
        axes.append(fig.add_subplot(1,2,2,adjustable='box',aspect=1))


    linex = np.arange(1,nx+1)*dx
    liney = np.arange(1,ny+1)*dy
    im0 = axes[0].imshow(np.flipud(np.fliplr(-s0)), extent=[0, linex[-1], 0, liney[-1]], vmin=vmin[0], vmax=vmax[0], 
                        cmap=plt.get_cmap(cmap_type))
    axes[0].set_title('(a) {0}'.format(tit0), loc='left')
    axes[0].set_aspect('equal')
    axes[0].set_xlabel('Offshore distance [m]')
    axes[0].set_ylabel('Alongshore distance [m]')
    im1 = axes[1].imshow(np.flipud(np.fliplr(-s1)), extent=[0, linex[-1], 0, liney[-1]], vmin=vmin[1], vmax=vmax[1], 
                   cmap=plt.get_cmap(cmap_type))
    axes[1].set_title('(b) Estimate'.format(tit1), loc='left')
    axes[1].set_xlabel('Offshore distance [m]')
    axes[1].set_aspect('equal')
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    if colorbar_flag==1:
        fig.colorbar(im1, cax=cbar_ax)
    else:
        fig.colorbar(im0, cax=cbar_ax)



    return fig,axes


In [ ]:
fig, axes = plot_2x1(nx,ny,dx[0],dx[1],s_true2d,s_hat2d,cmap_type='jet')

In [ ]:
fig, axes = plot_2x1(nx,ny,dx[0],dx[1],s_hat2d,error,tit0='Estimate',tit1='Error',colorbar_flag=1,
                    vmin=[-7.,error.min().min()],vmax=[0.,error.max().max()])

In [ ]:
fig, axes = plot_1x1(nx,ny,dx[0],dx[1],error,tit0='Error',
                    vmin=[error.min().min()],vmax=[error.max().max()],cmap_type='coolwarm')

In [ ]:
def get_transect(nx,ny,dx,dy,s_true2d,post_std2d,ix):
    linex = np.arange(1,nx+1)*dx
    line1_true = s_true2d[ny-ix+1,:]
    line1 = s_hat2d[ny-ix+1,:]
    line1_u = s_hat2d[ny-ix+1,:] + 1.96*post_std2d[ny-ix+1,:]
    line1_l = s_hat2d[ny-ix+1,:] - 1.96*post_std2d[ny-ix+1,:]

    return linex,line1_true,line1,line1_u,line1_l



In [ ]:
def plot_transect(nx,ny,dx,dy,s_true2d,post_std2d,ix,axes=None,linewidth=3):
    if axes is None:
        axes = plt.gca()
    linex,line1_true,line1,line1_u,line1_l = get_transect(nx,ny,dx,dy,s_true2d,post_std2d,ix)
    axes.plot(linex, np.flipud(-line1_true),'r-', label='True',linewidth=linewidth)
    axes.plot(linex, np.flipud(-line1),'k-', label='Estimated',linewidth=linewidth)
    axes.plot(linex, np.flipud(-line1_u),'k--', label='95% credible interval',linewidth=linewidth)
    axes.plot(linex, np.flipud(-line1_l),'k--',linewidth=linewidth)
    #axes.plot(linex, np.flipud(-line1_X),'b--', label='Drift/Trend')
    axes.set_title('(a) {0:4.1f} m'.format(ix*dy), loc='left')
    handles, labels = axes.get_legend_handles_labels()
    axes.legend(handles, labels)
    axes.set_xlabel('Offshore distance [m]')
    axes.set_ylabel('z [m]')
    return axes

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1,adjustable='box',aspect=20)
ax = plot_transect(nx,ny,dx[0],dx[1],s_true2d,post_std2d,25,axes=ax)


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1,adjustable='box',aspect=20)
ax = plot_transect(nx,ny,dx[0],dx[1],s_true2d,post_std2d,45,axes=ax)



In [ ]:
fig = plt.figure(figsize=(10,10))

fig.suptitle('transect with prior var.: (%g)^2, n_pc : %d, lx = %f m, ly = %f m\n\n' % (prior_std, params['n_pc'],prior_cov_scale[0],prior_cov_scale[1]))

linex,line1_true,line1,line1_u,line1_l = get_transect(nx,ny,dx[0],dx[1],s_true2d,post_std2d,25)
linex,line2_true,line2,line2_u,line2_l = get_transect(nx,ny,dx[0],dx[1],s_true2d,post_std2d,45)
axes = []
axes.append(fig.add_subplot(2,1,1,adjustable='box',aspect=20))
axes[0] = plot_transect(nx,ny,dx[0],dx[1],s_true2d,post_std2d,25,axes=axes[0])

axes.append(fig.add_subplot(2,1,2,adjustable='box',aspect=20))
axes[1] = plot_transect(nx,ny,dx[0],dx[1],s_true2d,post_std2d,45,axes=axes[1])



In [ ]:
nobs = obs.shape[0]
fig = plt.figure()
plt.title('obs. vs simul.')
plt.plot(obs,simul_obs,'.',markersize=4)
plt.xlabel('observation')
plt.ylabel('simulation')
minobs = np.vstack((obs,simul_obs)).min().min()
maxobs = np.vstack((obs,simul_obs)).max().max()
plt.plot(np.linspace(minobs,maxobs,20),np.linspace(minobs,maxobs,20),'k-',linewidth=3)
plt.axis('equal')
axes = plt.gca()
axes.set_xlim([math.floor(minobs),math.ceil(maxobs)])
axes.set_ylim([math.floor(minobs),math.ceil(maxobs)])